## Постановка задачи
Загрузим данные, приведем их к числовым, заполним пропуски, нормализуем данные и оптимизируем память.

Разделим выборку на обучающую/проверочную в соотношении 80/20.

Построим несколько моделей дерева решений, найдем оптимальную через перекрестную валидацию (CV).

Проведем предсказание и проверим качество через каппа-метрику.

Данные:
* https://video.ittensive.com/machine-learning/prudential/train.csv.gz

Соревнование: https://www.kaggle.com/c/prudential-life-insurance-assessment/

© ITtensive, 2020

In [1]:
GRAIN = 11
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import cohen_kappa_score, confusion_matrix, make_scorer
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn import preprocessing
from IPython.display import SVG, display
from graphviz import Source
from etl_utils import reduce_mem_usage, show_inf_and_na, inf_and_na_columns
import os


os.environ["PATH"] += (os.pathsep + 'C:/Program Files/Graphviz/bin/')
pd.set_option('display.max_columns', 200)

data = pd.read_csv("https://video.ittensive.com/machine-learning/prudential/train.csv.gz")

data['Product_Info_2_1'] = data['Product_Info_2'].str.slice(0, 1)
data['Product_Info_2_2'] = pd.to_numeric(data['Product_Info_2'].str.slice(1, 2))
data = data.drop('Product_Info_2', axis='columns')

onehot_df = pd.get_dummies(data['Product_Info_2_1'])
onehot_df.columns = ['Product_Info_2_1' + column for column in onehot_df.columns]
data = pd.merge(left=data, right=onehot_df, left_index=True, right_index=True).drop('Product_Info_2_1', axis=1).fillna(-1)
del onehot_df

### Набор столбцов для расчета
"Облегченный" вариант для визуализации дерева

In [2]:
columns = ["Wt", "Ht", "Ins_Age", "BMI"]

### Нормализация данных

In [3]:
scaler = preprocessing.StandardScaler()
data_transformed = pd.DataFrame(scaler.fit_transform(data[columns]))
columns_transformed = data_transformed.columns
data_transformed['Response'] = data['Response']
data_transformed = reduce_mem_usage(data_transformed)
data_transformed.info()

Потребление памяти меньше на 1.76 Мб (-77.5%)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59381 entries, 0 to 59380
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   0         59381 non-null  float16
 1   1         59381 non-null  float16
 2   2         59381 non-null  float16
 3   3         59381 non-null  float16
 4   Response  59381 non-null  int8   
dtypes: float16(4), int8(1)
memory usage: 522.0 KB


### Разделение данных
Преобразуем выборки в отдельные наборы данных

In [4]:
data_train, data_test = train_test_split(data_transformed, test_size=.2, random_state=GRAIN)
data_train.head()

,0,1,2,3,Response
40387,-0.090637,1.249023,-0.618652,-0.786133,8
17090,-0.513672,-0.220581,0.062622,-0.481445,7
21687,0.426270,1.003906,1.046875,-0.081970,8
40073,0.567383,1.003906,1.878906,0.075378,6
3408,-0.395996,-0.220581,0.516602,-0.329834,7


### Дерево решений
Минимальное число "одинаковых" значений для ветвления - 10

In [5]:
x = data_train[columns_transformed]
model = DecisionTreeClassifier(random_state=GRAIN, min_samples_leaf=10).fit(x, data_train['Response'])

### Визуализация дерева
Доступно несколько форматов вывода, большой SVG выводится в Jupyter Notebook сложнее, поэтому используем PNG.

В качестве названий параметров передаем исходный список.

In [6]:
graph = Source(export_graphviz(
    model, out_file=None, feature_names=columns, filled=True,
    class_names=data_train['Response'].unique().astype('str')
))

with open('tree.png', 'wb') as f:
    f.write(graph.pipe(format='png'))

dot: graph is too large for cairo-renderer bitmaps. Scaling by 0.101773 to fit


### Влияние признаков
Выведем долю влияния признаков на конечный ответ в дерева

In [7]:
model.feature_importances_

array([0.14940619, 0.03979557, 0.26352872, 0.54726951])

### Перекрестная проверка (CV)
Разбиваем обучающую выборку еще на k (часто 5) частей, на каждой части данных обучаем модель. Затем проверяем 1-ю, 2-ю, 3-ю, 4-ю части на 5; 1-ю, 2-ю, 3-ю, 5-ю части на 4 и т.д.

В итоге обучение пройдет весь набор данных, и каждая часть набора будет проверена на всех оставшихся (перекрестным образом).

Перекрестная проверка используется для оптимизации параметров исходной модели - решающего дерева в данном случае. Зададим несколько параметров для перебора и поиска самой точной модели.

Для проверки будем использовать каппа-метрику.

In [8]:
trea_params = {
    'max_depth': range(10, 20),
    'max_features': range(1, len(columns_transformed)),
    'min_samples_leaf': range(20, 100)
}

tree_grid = GridSearchCV(model, trea_params, cv=5, n_jobs=-1, verbose=True, scoring=make_scorer(cohen_kappa_score))
tree_grid.fit(x, data_train['Response'])

Fitting 5 folds for each of 2400 candidates, totalling 12000 fits


GridSearchCV(cv=5,
             estimator=DecisionTreeClassifier(min_samples_leaf=10,
                                              random_state=11),
             n_jobs=-1,
             param_grid={'max_depth': range(10, 20),
                         'max_features': range(1, 4),
                         'min_samples_leaf': range(20, 100)},
             scoring=make_scorer(cohen_kappa_score), verbose=True)

Выведем самые оптимальные параметры и построим итоговую модель

In [9]:
print(tree_grid.best_params_)
model = DecisionTreeClassifier(
    random_state=GRAIN,
    min_samples_leaf=tree_grid.best_params_['min_samples_leaf'],
    max_features=tree_grid.best_params_['max_features'],
    max_depth=tree_grid.best_params_['max_depth']
)
model.fit(x, data_train['Response'])

{'max_depth': 10, 'max_features': 3, 'min_samples_leaf': 65}


DecisionTreeClassifier(max_depth=10, max_features=3, min_samples_leaf=65,
                       random_state=11)

### Предсказание данных и оценка модели

In [10]:
data_test['target'] = model.predict(data_test[columns_transformed])

Кластеризация дает 0.192, kNN(100) - 0.3, лог. регрессия - 0.512/0.496, SVM - 0.95

In [11]:
print("Решающее дерево:", round(cohen_kappa_score(data_test['target'], data_test['Response'], weights='quadratic'), 3))

Решающее дерево: 0.289


### Матрица неточностей

In [12]:
print("Решающее дерево:\n", confusion_matrix(data_test["target"], data_test["Response"]))

Решающее дерево:
 [[ 149   80    4    4   35  102   71   63]
 [  92  172    6    2   59   32   15   10]
 [   0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0]
 [ 117  150   38    0  374  141   11    8]
 [ 269  284   52   38  236  736  370  213]
 [ 131  122   17   25   88  266  289  173]
 [ 480  499   85  247  272  959  806 3485]]
